<a href="https://colab.research.google.com/github/YuliiaChorna1/Data-Science-11.2-Tensorflow-Graphs-Neural-Networks-Auto-differentiation/blob/main/computational_graph_practice_extralesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational graph / обчислювальний граф

https://www.tensorflow.org/guide/intro_to_graphs

In [2]:
import tensorflow as tf


def formula(x: tf.Tensor, y: tf.Tensor, b: tf.Tensor):
    x = tf.matmul(x, y)
    x = x + b
    return x

# Функція tf.function приймає у якості аргумента звичайну функцію
# і генерує обчислювальний граф:
function_that_uses_graph = tf.function(formula)

# Ми будемо виконувати матричні операції,
# тому перш ніж передавати аргументи на цю функцію потрібно перетворити їх на тензори:
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

orig_value = formula(x1, y1, b1).numpy()

# Після виконання обчислень функція function_that_uses_a_graph поверне тензор,
# який буде перетворено на багатовимірний масив numpy:
tf_function_value = function_that_uses_graph(x1, y1, b1).numpy()

assert(orig_value == tf_function_value)

In [3]:
def my_func(x, y):
    # A simple hand-rolled layer
    return tf.nn.relu(tf.matmul(x, y))

my_func_tf = tf.function(my_func)

my_func_tf(x1, y1).numpy()

array([[8.]], dtype=float32)

In [8]:
# This is the graph-generating output of Autograph
print(tf.autograph.to_code(my_func_tf.python_function))

def tf__my_func(x, y):
    with ag__.FunctionScope('my_func', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        try:
            do_return = True
            retval_ = ag__.converted_call(ag__.ld(tf).nn.relu, (ag__.converted_call(ag__.ld(tf).matmul, (ag__.ld(x), ag__.ld(y)), None, fscope),), None, fscope)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [6]:
# This is the graph itself
print(my_func_tf.get_concrete_function(x1, y1).graph.as_graph_def())

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1
        }
        dim {
          size: 2
        }
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
}
node {
  name: "y"
  op: "Placeholder"
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 2
        }
        dim {
          size: 1
        }
      }
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "_user_specified_name"
    value {
      s: "y"
    }
  }
}
node {
  name: "MatMul"
  op: "MatMul"
  input: "x"
  input: "y"
  attr {
    key: "transpose_b"
    value {
      b: false
    }
  }
  attr {
    key: "transpose_a"
    value {
      b: false
    }
  }
  attr {
    key: "grad_b"
    value {
      b: false
    }
  }
  attr {
    key: "grad_a"
    value {
      b: false

https://www.tensorflow.org/tensorboard/graphs#graphs_of_tffunctions

In [ ]:
from datetime import datetime
import tensorflow as tf

# The function to be traced.
@tf.function
def my_func(x, y):
    # A simple hand-rolled layer.
    return tf.nn.relu(tf.matmul(x, y))

# Set up logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp
writer = tf.summary.create_file_writer(logdir)

# Sample data for your function.
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = my_func(x, y)
with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace",
        step=0,
        profiler_outdir=logdir)

# tf.summary.trace_off() # Explicitly stop tracing to ensure clean up

In [ ]:
! tensorboard --logdir logs/func

Швидкість виконання графа в TensorFlow порівняно з інтерактивним виконанням в основному залежить від наступних факторів:

- Оптимізація графа: Під час виконання графа TensorFlow може оптимізувати та перетворювати граф обчислень перед його виконанням. Цей процес оптимізації може включати усунення спільних підвиразів, складання констант та інші оптимізації на рівні графа.
- Розміщення пристроїв та паралельне виконання: Графічне виконання TensorFlow дозволяє ефективно розміщувати пристрої та виконувати обчислення паралельно. Граф обчислень може бути розділений на підграфи, які можуть виконуватися паралельно на кількох пристроях, таких як GPU або TPU.
- Компіляція у прискорений код: Коли використовується графічне виконання, TensorFlow має можливість компілювати граф обчислень в більш оптимізовану форму на низькому рівні, специфічні для певного обладнання.
- Зменшення накладень Python: Інтерактивне виконання включає в себе виконання операцій імперативно, що означає, що кожна операція виконується негайно в Python. Це може вводити накладення Python та обмежувати можливості оптимізації. На відміну від цього, під час графічного виконання граф будується та оптимізується перед виконанням, що зменшує необхідність у частих взаємодіях із інтерпретатором Python.

In [4]:
import timeit
import tensorflow as tf

x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
    result = tf.eye(10, dtype=tf.dtypes.int32)
    for _ in range(y):
        result = tf.matmul(x, result)
    return result

print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000), "seconds")

Eager execution: 5.836813345999872 seconds


In [5]:
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000), "seconds")

Graph execution: 1.0712397730001157 seconds
